In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import pandas as pd
df = pd.read_csv(r"dataset//names.csv")

In [3]:
df.head()

,gender,eng,count,pr_fa,abjad,pr_en,origin,civil_registration_approved
0,دختر,sana,76899.0,ثنا,551.0,sanā,عربی,تایید شده
1,دختر,samin,25615.0,ثمين,600.0,samin,عربی,تایید شده
2,دختر,sorayya,159065.0,ثريا,711.0,sorayyā,عربی,تایید شده
3,دختر,samar,2294.0,ثمر,740.0,samar,عربی,تایید شده
4,دختر,samina,2091.0,ثمينا,601.0,saminā,"فارسی , عربی",تایید شده


In [4]:
len(df)

8888

In [5]:
names = df["pr_fa"]

In [6]:
names.isna().sum(axis=0)

9

In [7]:
names.shape

(8888,)

In [8]:
names = names.dropna()

In [9]:
# chars = sorted(list(set(''.join(names))))

In [10]:
chars = list(set(''.join(names)))
ctoi = {c:i+1 for i, c in enumerate(chars)}
ctoi['.'] = 0
itoc = {i:c for c,i in ctoi.items()}

In [11]:
print(ctoi)

{'ا': 1, 'ک': 2, 'ؤ': 3, 'چ': 4, 'ع': 5, 'ش': 6, 'ه': 7, 'ف': 8, ' ': 9, 'آ': 10, 'ظ': 11, 'گ': 12, 'ذ': 13, 'ت': 14, 'ض': 15, 'م': 16, 'ق': 17, 'ز': 18, 'ة': 19, 'ث': 20, 'و': 21, 'ی': 22, 'ء': 23, 'س': 24, 'ك': 25, 'ل': 26, 'ح': 27, 'غ': 28, 'ص': 29, 'ج': 30, 'ژ': 31, 'ن': 32, 'پ': 33, 'ر': 34, 'ط': 35, 'ب': 36, 'خ': 37, 'ي': 38, 'د': 39, 'ئ': 40, 'أ': 41, '.': 0}


In [12]:
block_size = 5
X, Y = [], []
count = 0
for n in names :
    n = n + '.'
    x = block_size * [0]
    for ch in n :
        count = count + 1
        ix = ctoi[ch]
        X.append(x)
        Y.append(ix)
        x = x[1:] + [ix]
        
    print(x)
    print(''.join(itoc[c] for c in x))

[0, 20, 32, 1, 0]
.ثنا.
[20, 16, 38, 32, 0]
ثمين.
[20, 34, 38, 1, 0]
ثريا.
[0, 20, 16, 34, 0]
.ثمر.
[16, 38, 32, 1, 0]
مينا.
[20, 1, 16, 34, 0]
ثامر.
[16, 38, 32, 7, 0]
مينه.
[20, 1, 16, 32, 0]
ثامن.
[21, 36, 1, 32, 0]
وبان.
[20, 1, 17, 36, 0]
ثاقب.
[1, 26, 26, 7, 0]
الله.
[16, 38, 26, 7, 0]
ميله.
[1, 36, 14, 7, 0]
ابته.
[20, 16, 34, 7, 0]
ثمره.
[0, 20, 16, 32, 0]
.ثمن.
[32, 1, 38, 1, 0]
نايا.
[16, 38, 26, 1, 0]
ميلا.
[1, 26, 26, 7, 0]
الله.
[1, 32, 22, 7, 0]
انیه.
[20, 34, 21, 14, 0]
ثروت.
[32, 1, 12, 21, 0]
ناگو.
[21, 22, 36, 7, 0]
ویبه.
[16, 1, 32, 7, 0]
مانه.
[10, 34, 1, 39, 0]
آراد.
[34, 14, 38, 32, 0]
رتين.
[10, 24, 32, 1, 0]
آسنا.
[21, 38, 32, 1, 0]
وينا.
[22, 7, 1, 32, 0]
یهان.
[10, 34, 38, 1, 0]
آريا.
[34, 24, 1, 16, 0]
رسام.
[0, 10, 21, 1, 0]
.آوا.
[34, 21, 38, 32, 0]
روين.
[10, 32, 22, 1, 0]
آنیا.
[10, 21, 38, 32, 0]
آوين.
[10, 34, 1, 18, 0]
آراز.
[34, 2, 1, 32, 0]
رکان.
[32, 38, 24, 1, 0]
نيسا.
[10, 34, 14, 1, 0]
آرتا.
[34, 16, 38, 32, 0]
رمين.
[38, 26, 38, 32, 0]
يلين.
[10

In [13]:
import torch
X = torch.tensor(X)
Y = torch.tensor(Y)

In [14]:
NeuronCount1 = 2
charsLen = len(ctoi)

In [15]:
device = torch.device('cuda')

W0 = torch.randn(charsLen, NeuronCount1) * 0.1
W0 = W0.to(device)

NeuronCount2 = 200
W1 = torch.randn((NeuronCount1 * block_size, NeuronCount2)) * 0.1
W1 = W1.to(device)
b1 = torch.randn(NeuronCount2)
b1 = b1.to(device)

emb = W0[X]
emb = emb.to(device)
emb = emb.view(-1, block_size * NeuronCount1)
emb = emb.to(device)

h = torch.tanh(emb @ W1 + b1)
h = h.to(device)

W2 = torch.randn((NeuronCount2, charsLen)) * 0.1
W2 = W2.to(device)
b2 = torch.randn(charsLen) 
b2 = b2.to(device)

parameters = [W0, W1, b1, W2, b2]

for p in parameters :
    p.requires_grad = True

lr = 4e-1

for i in range(10000) :
    # forward pass
    emb = W0[X]
    
    h = torch.tanh(emb.view(-1, block_size * NeuronCount1) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y.to(device))
    
    # backward pass
    for p in parameters :
        p.grad = None
    loss.backward()
    
    if i % 100 == 0:
        print(loss.item())
    
    # update
    for p in parameters :
        p.data -= lr * p.grad

4.506479740142822
2.929518699645996
2.7324230670928955
2.7042462825775146
2.6500442028045654
2.620190382003784
2.6017420291900635
2.5898070335388184
2.580216646194458
2.569178581237793
2.5577316284179688
2.54451847076416
2.532630205154419
2.5212457180023193
2.5097577571868896
2.501269578933716
2.490940809249878
2.4838502407073975
2.473318338394165
2.4702885150909424
2.465984582901001
2.458728075027466
2.4550347328186035
2.449019193649292
2.434597969055176
2.4357686042785645
2.423038959503174
2.4204204082489014
2.411407470703125
2.403113603591919
2.401496648788452
2.3948721885681152
2.391219139099121
2.387359142303467
2.384232759475708
2.377009153366089
2.372690200805664
2.3700149059295654
2.3678665161132812
2.3608756065368652
2.356055974960327
2.353034496307373
2.353307008743286
2.345747709274292
2.3410332202911377
2.33919620513916
2.3364346027374268
2.330920457839966
2.3307180404663086
2.3234548568725586
2.3205740451812744
2.318037748336792
2.3161089420318604
2.3136940002441406
2.3108

In [16]:
import matplotlib.pyplot as plt

plt.imshow(W2.T.detach().numpy(), cmap='gray')

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
import random
import torch
import torch.nn.functional as F

def build_name():
    word = []

    ix = random.randint(1, charsLen-1)  
    ixs = [0] * block_size
    ixs[-1] = ix 
    while ix != 0:
        x = F.one_hot(torch.tensor(ixs), num_classes=charsLen).float().to(device)  
        emb = x @ W0                                                   
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)                      
        logits = h @ W2 + b2                                           
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdim=True)
        probs = probs.view(-1)
        # print(f"Probs: {probs}")
        word.append(itoc[ix])
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item()
        ixs = ixs[1:] + [ix]

    return word

In [25]:
for i in range(10):
    print('' .join(build_name()))

مهنوش
ؤحيقاط
گلوشه
ذريما
هيشا
پسارچسلا
ساشقاد
دانوی
حعار
ظريفه
